# Feature 7: ocasiones de gol creadas p90 (asistencias a remates)

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
laLiga = pd.read_csv(r'C:\Users\DELUX\Desktop\datos_OPTA\laLiga.csv')

In [4]:
jugadas = laLiga[['id_jugador','categoria_evento','fecha','id_evento','id_periodo','minuto','segundo','equipo','calificador','valor','jugador','x','y','outcome']].copy()
jugadas['valor'].fillna(True, inplace=True)

In [5]:
jugadas.drop(jugadas.index[[177812]], inplace=True)

In [6]:
jugadas = jugadas.set_index(['id_jugador','categoria_evento','fecha','id_evento','id_periodo','minuto','segundo','equipo','jugador','x','y','outcome','calificador'])['valor'].unstack()
jugadas = jugadas.reset_index().rename_axis(None).rename_axis(None, axis=1)
jugadas.fillna(False, inplace=True)

In [7]:
jugadas = jugadas[jugadas['jugador']!=False]

In [8]:
mtr7 = jugadas[(jugadas['asistencia a remate']==True)&(jugadas['outcome']==1)]

In [9]:
len(mtr7)

6456

In [10]:
mtr7 = mtr7[['fecha','id_jugador','jugador','equipo']]

In [11]:
mtr7.head()

,fecha,id_jugador,jugador,equipo
11294,5,10316,Joaquín Sánchez Rodríguez,Real Betis
11341,7,10316,Joaquín Sánchez Rodríguez,Real Betis
11359,7,10316,Joaquín Sánchez Rodríguez,Real Betis
11363,7,10316,Joaquín Sánchez Rodríguez,Real Betis
11374,7,10316,Joaquín Sánchez Rodríguez,Real Betis


In [12]:
mtr7 = mtr7.groupby(['id_jugador','jugador','equipo']).count().rename(columns={'fecha':'mtr7'}).reset_index()

In [13]:
mtr7.head()

,id_jugador,jugador,equipo,mtr7
0,10316.0,Joaquín Sánchez Rodríguez,Real Betis,44
1,11129.0,David López,Espanyol,4
2,11602.0,Borja Fernández,Real Valladolid,1
3,14028.0,Juan José Camacho Barnola,Huesca,1
4,14464.0,Aritz Aduriz,Athletic Club,7


In [14]:
mtr7.sort_values('mtr7',ascending=False).head()

,id_jugador,jugador,equipo,mtr7
16,19054.0,Lionel Messi,Barcelona,91
201,92190.0,Jonathan Rodríguez Menéndez,Alavés,84
64,51952.0,Daniel Parejo,Valencia CF,79
119,76650.0,Antoine Griezmann,Atlético de Madrid,72
17,19524.0,Santiago Cazorla,Villarreal,70


In [15]:
mtr7[mtr7['jugador']=='Raúl García']

,id_jugador,jugador,equipo,mtr7
13,18498.0,Raúl García,Athletic Club,23
134,80908.0,Raúl García,Girona,6


In [16]:
dataAggJug = pd.read_csv(r'C:\Users\DELUX\Desktop\consultoria_futbol\comparativoJugadores\dataAggJug.csv')

In [17]:
dataAggJug.head()

,fecha,id_jugador,estadistica,num_estadistica,jugador
0,1,212769,diving_save,1,Unai Simón
1,1,212769,leftside_pass,2,Unai Simón
2,1,212769,accurate_pass,10,Unai Simón
3,1,212769,rightside_pass,1,Unai Simón
4,1,212769,attempts_conceded_ibox,9,Unai Simón


In [18]:
minJug = dataAggJug[dataAggJug['estadistica']=='mins_played']

In [19]:
minJug.head()

,fecha,id_jugador,estadistica,num_estadistica,jugador
34,1,212769,mins_played,90,Unai Simón
111,1,59062,mins_played,65,Óscar De Marcos
180,1,197319,mins_played,90,Yeray Álvarez
249,1,242831,mins_played,90,Peru Nolaskoain
328,1,42670,mins_played,90,Yuri Berchiche Izeta


In [20]:
minJug = minJug.drop('estadistica',1).rename(columns={'num_estadistica':'minJug'})

In [21]:
minJug.head()

,fecha,id_jugador,minJug,jugador
34,1,212769,90,Unai Simón
111,1,59062,65,Óscar De Marcos
180,1,197319,90,Yeray Álvarez
249,1,242831,90,Peru Nolaskoain
328,1,42670,90,Yuri Berchiche Izeta


In [22]:
minJug = minJug.groupby(['id_jugador','jugador']).sum().reset_index().drop('fecha',1)

In [23]:
minJug.head()

,id_jugador,jugador,minJug
0,7261,Daniele Bonera,233
1,10316,Joaquín Sánchez Rodríguez,1694
2,11129,David López,1820
3,11602,Borja Fernández,839
4,14028,Juan José Camacho Barnola,34


In [24]:
mtr7x90 = minJug.merge(mtr7,on=['id_jugador','jugador'],how='left') #Hacemos left join para tener tabla con minutos jugados por jugador por fecha
mtr7x90['mtr7x90'] = mtr7x90['mtr7']/mtr7x90['minJug']*90 #Creamos mtr2x90
mtr7x90 = mtr7x90.drop(['mtr7','minJug'],1)

In [25]:
mtr7x90.head()

,id_jugador,jugador,equipo,mtr7x90
0,7261,Daniele Bonera,NaN,NaN
1,10316,Joaquín Sánchez Rodríguez,Real Betis,2.337662
2,11129,David López,Espanyol,0.197802
3,11602,Borja Fernández,Real Valladolid,0.107271
4,14028,Juan José Camacho Barnola,Huesca,2.647059


In [26]:
mtr7x90.isnull().sum()

id_jugador     0
jugador        0
equipo        89
mtr7x90       89
dtype: int64

In [27]:
mtr7x90['mtr7x90'] = mtr7x90['mtr7x90'].replace(np.nan, 0)

In [28]:
mtr7x90.isnull().sum()

id_jugador     0
jugador        0
equipo        89
mtr7x90        0
dtype: int64

In [29]:
feat7 = mtr7x90.copy()
feat7 = feat7.rename(columns={'mtr7x90':'feat7'})

In [30]:
feat7.head()

,id_jugador,jugador,equipo,feat7
0,7261,Daniele Bonera,NaN,0.000000
1,10316,Joaquín Sánchez Rodríguez,Real Betis,2.337662
2,11129,David López,Espanyol,0.197802
3,11602,Borja Fernández,Real Valladolid,0.107271
4,14028,Juan José Camacho Barnola,Huesca,2.647059


In [31]:
feat7.isnull().sum()

id_jugador     0
jugador        0
equipo        89
feat7          0
dtype: int64

In [32]:
feat7.sort_values('feat7',ascending=False).head(10)

,id_jugador,jugador,equipo,feat7
539,481655,Martín Zubimendi,Real Sociedad,15.000000
215,89572,Denis Suárez,Barcelona,5.294118
526,451358,Kang-In Lee,Valencia CF,4.285714
477,227165,Abel Ruiz,Barcelona,4.285714
462,220182,Gorka Guruzeta,Athletic Club,4.000000
431,203151,Kevin Soni,Girona,3.461538
22,19054,Lionel Messi,Barcelona,3.022140
4,14028,Juan José Camacho Barnola,Huesca,2.647059
166,81033,Nicola Sansone,Villarreal,2.647059
58,44989,Toni Kroos,Real Madrid,2.586439


In [33]:
cd C:\Users\DELUX\Desktop\publicaciones\1_DataScoutingClustering\1_MedioCentroCreativo\df

C:\Users\DELUX\Desktop\publicaciones\1_DataScoutingClustering\1_MedioCentroCreativo\df


In [34]:
feat7.to_csv('feat7.csv',encoding='utf-8', index=False,decimal=',')